# Creating interactive maps with Leaflet and Folium 

### Install updates and dependencies for Earth Engine API and Python Client

This is a one time step.

This notebook needs to be run in a Python 2 Kernel.

We are going to update Conda, and install a few requirements for running geospatial data in the notebook.

In [ ]:
%%bash
source activate ipykernel_py2 && \
conda update -n base conda && \
conda install -y -c conda-forge google-api-python-client earthengine-api pandas folium nodejs cython gdal

In [ ]:
%%bash
pip install --upgrade pip && \
pip install vega_datasets && \
pip install pyCrypto && \
pip install pathlib && \
pip install kml2geojson

In [ ]:
%%bash
jupyter labextension install @jupyterlab/geojson-extension

After these dependencies have installed, you should be ready to go!

## Authenticating to the Earth Engine servers

In [ ]:
%%bash
earthengine authenticate --quiet

Once you have obtained an authorization code from the previous step, paste the code into the following cell and run it:

In [ ]:
%%bash
earthengine authenticate --authorization-code=4/AABEZqqZsujlVJUO8LzoSsMa6GqKhqGsq_n_M0HwOyrOWMasjDJev8Y

## Import Python packages

In [ ]:
from oauth2client import crypt

In [ ]:
# import packages
import pandas as pd
import json
import sys
import ee 

ee.Initialize()

import folium
print(folium.__version__)
sys.path.append('..')
print (folium.__file__)
print (folium.__version__)

### Create definition files for working with Google Earth Engine

In [ ]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)


### Initiate your connection to the CyVerse DataStore with iRODS iCommands

This little script _MUST_ be run on a secure connection, else your password will be visible to anyone monitoring your work.

If you're not comfortable entering your CyVerse password in the notebook, you can open a terminal session and type `iinit`

You will be asked for your password there.

In [ ]:
import getpass
import os

_password = getpass.getpass()
command = "iinit" # can be any command but don't forget -S as it enables input from stdin
os.popen(command, 'w').write(_password+'\n') # newline char is important otherwise prompt will wait for you to manually perform newline
del(_password) # deletes your password on the notebook

Now, we want to copy our data to a new scratch directory. If your user does not own the directory you will have to also use `chmod` to change ownership.

```
sudo mkdir -p /scratch/2016_Campaign/HARV/L1/DiscreteLidar
sudo chown $USER:iplant-everone /scratch/ -R
```

Note, you will have to use `sudo` for directories your username does not own.

In [ ]:
import getpass
import os
_password = getpass.getpass()
command = "sudo chown $USER:$iplant-everyone /scratch/ -R" # can be any command but don't forget -S as it enables input from stdin
os.popen(command, 'w').write(_password+'\n') # newline char is important otherwise prompt will wait for you to manually perform newline
del(_password) # deletes your password on the notebook

Import Santa Rita Experimental Range Data

In [ ]:
%%bash
mkdir -p /scratch/2017_Campaign/SRER/L1/

In [ ]:
%%bash
iget -Pbrv /iplant/home/shared/NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary /scratch/2017_Campaign/SRER/L1

Convert the .kml to a .geojson file.

In [ ]:
%%python3
import kml2geojson
kml2geojson.main.convert("/scratch/2017_Campaign/SRER/L1/LasBoundary/full_boundary.kml", "/scratch/2017_Campaign/SRER/L1/LasBoundary/")

In [ ]:
# import geo json data
import json
full_boundary = json.load(open('/scratch/2017_Campaign/SRER/L1/LasBoundary/full_boundary.geojson'))

In [ ]:
# Get an area to look at
lat = 31.800
lon = -110.865
zoom_start=11

# Open Street Map Base
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)

# Add polygon outline of HARV
full_boundary = json.load(open('/scratch/2017_Campaign/SRER/L1/LasBoundary/full_boundary.geojson'))
folium.GeoJson(full_boundary).add_to(m)

# Add GEE Terrain Layer
image = ee.Image('srtm90_v4')
vis_params = {'min':0.0, 'max':3000, 'palette':'00FFFF,0000FF'}
folium_gee_layer(m,image,vis_params=vis_params,folium_kwargs={'overlay':True,'name':'SRTM'})

# Create a reference to the LANDSAT 8 image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
# Filter the LANDSAT collection down to a eight month period
filtered = l8.filterDate('2016-05-01', '2016-09-30');
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.median()
l8_vis_params = {'min': 0, 'max':0.3, 'bands':'B4,B3,B2'}
folium_gee_layer(m,l8_image,l8_vis_params,folium_kwargs={'overlay':True,'name':'LANDSAT'})

# Create a reference to the 2017 SRER NEON orthophotography image collection
neon = ee.ImageCollection('users/gponce/usda_ars/image_collections/neon_srer_2017_rgb')
neon_image = neon.median()
neon_vis_params = {'min':0, 'max':255, 'bands':'b1,b2,b3'}
folium_gee_layer(m,neon_image,neon_vis_params,folium_kwargs={'overlay':True,'name':'NEON'})

# Create a reference to the 2016 sUAS orthophotography imager collection

m.add_child(folium.LayerControl())
m

Import layer from Walnut Gulch

In [ ]:
%%bash
iget -Pvrf /iplant/home/shared/srer-wgew/data/wgew/sfm_2017/LH_20agl_9Oct2017_georef.tif /scratch/wgew

In [3]:
%%python3
import gdal as gdal
import osr as osr

gdal.UseExceptions()

fname = '/scratch/wgew/LH_20agl_9Oct2017_georef.tif'

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)

PROJCS["NAD_1983_UTM_Zone_12N",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-111],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AUTHORITY["EPSG","26912"]]


In [5]:
%%bash
gdalwarp -t_srs EPSG:4326 -r near /scratch/wgew/LH_20agl_9Oct2017_georef.tif /scratch/wgew/LH_20agl_9Oct2017_georef2.tif

Creating output file that is 39200P x 28406L.
Processing /scratch/wgew/LH_20agl_9Oct2017_georef.tif [1/1] : 0Using internal nodata values (e.g. 256) for image /scratch/wgew/LH_20agl_9Oct2017_georef.tif.
Copying nodata values from source /scratch/wgew/LH_20agl_9Oct2017_georef.tif to destination /scratch/wgew/LH_20agl_9Oct2017_georef2.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
%%bash
gdal_translate --config GDAL_CACHEMAX 4096 -of mbtiles  /scratch/wgew/LH_20agl_9Oct2017_georef2.tif /scratch/wgew/LH_20agl_9Oct2017_georef.mbtiles
gdaladdo -r nearest /scratch/wgew/LH_20agl_9Oct2017_georef.mbtiles 2 4 8 16 